In [2]:
!pip install ollama tiktoken
!pip install --force-reinstall urllib3
!pip install -U langchain-ollama

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.52 requires beautifulsoup4>=4.11.1, but you have beautifulsoup4 4.9.3 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Applications/Xcode.app/Cont

In [2]:
import ollama

for chunk in ollama.chat(model="llama3.2", messages=[
    {"role": "user", "content": "Tell me a short story."}
], stream=True):
    print(chunk["message"]["content"], end="", flush=True)

Once upon a time, in a small village nestled between two great mountains, there lived an old woodcarver named Kaito. He was known throughout the village for his incredible skill and attention to detail, which allowed him to bring even the most delicate of creatures to life.

One day, a young girl named Akira wandered into Kaito's workshop, her eyes wide with wonder as she took in the array of half-finished sculptures and half-carved logs that lined the walls. She had heard stories about the old man's magic, how he could shape wood into anything his heart desired.

Akira approached Kaito, who was sitting in a worn armchair by the fire, his eyes fixed on a small wooden bird that sat on his lap. "Master Kaito," she said, her voice barely above a whisper, "I've heard you can create magic with wood."

Kaito looked up at Akira, a hint of a smile on his face. "Ah, yes," he said. "Wood is a powerful tool. With the right touch and the heart of the craftsman, it can become anything."

Without wa

In [3]:
import os

def delete_sumy_files(directory):
    """
    Recursively traverses the given directory and deletes all .sumy files.
    
    Args:
        directory (str): The path to the repository.
    """
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.sumy'):
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Error deleting {file_path}: {e}")

if __name__ == "__main__":
    # Replace 'path/to/your/repo' with the actual path of your repository.
    repo_path = "/Users/reva/Documents/geek_projects/dungbeetle"
    delete_sumy_files(repo_path)

In [2]:
import os
import ollama
import time
import sys
from prompts import (
    GENERAL_SUMMARY_PROMPT_TEMPLATE,
    FILE_SUMMARY_PROMPT_TEMPLATE,
    FOLDER_SUMMARY_PROMPT_TEMPLATE,
    ROOT_PROJECT_SUMMARY_PROMPT_TEMPLATE
)

def generate_summary(content, context="code"):
    """
    Uses the locally running Llama 3.2 model via Ollama to summarize the given content.
    """
    prompt = GENERAL_SUMMARY_PROMPT_TEMPLATE.format(context=context, content=content)
    result = ""
    for chunk in ollama.chat(
        model="llama3.2", 
        messages=[{"role": "user", "content": prompt}],
        stream=True
    ):
        result += chunk["message"]["content"]
    return result.strip()

def recursive_summarize_text(text, context="code", max_length=131072):
    """
    Recursively summarizes the given text so that the final summary does not exceed max_length.
    If the text exceeds max_length, it splits it into chunks, summarizes each,
    then recursively summarizes the combined summaries until the result is short enough.
    """
    if len(text) <= max_length:
        return generate_summary(text, context)
    else:
        chunks = [text[i:i+max_length] for i in range(0, len(text), max_length)]
        partial_summaries = [recursive_summarize_text(chunk, context, max_length) for chunk in chunks]
        combined = "\n".join(partial_summaries)
        return recursive_summarize_text(combined, context, max_length)

def generate_file_summary(file_name, content):
    """
    Generates a structured summary for an individual file.
    """
    prompt = FILE_SUMMARY_PROMPT_TEMPLATE.format(file_name=file_name, content=content)
    return recursive_summarize_text(prompt, context="file structured")

def generate_folder_summary(folder_path, file_summaries, subfolder_summaries):
    """
    Generates a summary for a non-root folder by combining file summaries and subfolder summaries.
    """
    combined_context = ""
    for fname, summary in file_summaries.items():
        combined_context += f"File Summary - {fname}:\n{summary}\n\n"
    for subfolder, summary in subfolder_summaries.items():
        combined_context += f"Subfolder Summary - {subfolder}:\n{summary}\n\n"
    
    prompt = FOLDER_SUMMARY_PROMPT_TEMPLATE.format(folder_path=folder_path, context=combined_context)
    return recursive_summarize_text(prompt, context="folder structured")

def generate_root_project_summary(folder_path, combined_context):
    """
    Generates the root-level project summary.
    """
    prompt = ROOT_PROJECT_SUMMARY_PROMPT_TEMPLATE.format(folder_path=folder_path, context=combined_context)
    return recursive_summarize_text(prompt, context="project summary")

def traverse_and_summarize(folder_path, repo_path):
    """
    Recursively traverses the repository in a bottom-up manner.
    Skips any folder whose name starts with a dot (e.g. .git, .github).
    For each folder, it generates a summary by combining the summaries of files
    and immediate subfolders, then writes the summary to a .sumy file.
    """
    subfolder_summaries = {}
    
    # Process subdirectories first (skip hidden folders)
    for entry in os.listdir(folder_path):
        full_path = os.path.join(folder_path, entry)
        if os.path.isdir(full_path) and not os.path.basename(full_path).startswith('.'):
            traverse_and_summarize(full_path, repo_path)
            subfolder_name = os.path.basename(full_path)
            summary_file = os.path.join(full_path, f"{subfolder_name}.sumy")
            if os.path.exists(summary_file):
                with open(summary_file, "r", encoding="utf-8") as f:
                    subfolder_summaries[entry] = f.read()
            else:
                subfolder_summaries[entry] = "No summary available."
    
    file_summaries = {}
    # Process files in the current folder (ignoring .sumy files)
    for entry in os.listdir(folder_path):
        full_path = os.path.join(folder_path, entry)
        if os.path.isfile(full_path) and not entry.endswith(".sumy"):
            try:
                with open(full_path, "r", encoding="utf-8", errors="ignore") as f:
                    content = f.read()
                file_summaries[entry] = generate_file_summary(entry, content)
            except Exception as e:
                file_summaries[entry] = f"Error reading file: {e}"
    
    combined_context = ""
    for fname, summary in file_summaries.items():
        combined_context += f"File Summary - {fname}:\n{summary}\n\n"
    for subfolder, summary in subfolder_summaries.items():
        combined_context += f"Subfolder Summary - {subfolder}:\n{summary}\n\n"
    
    folder_name = os.path.basename(os.path.normpath(folder_path)) or "root"
    
    # For the repository root, use the project-level structured summary
    if os.path.abspath(folder_path) == os.path.abspath(repo_path):
        final_summary = generate_root_project_summary(folder_path, combined_context)
    else:
        final_summary = generate_folder_summary(folder_path, file_summaries, subfolder_summaries)
    
    summary_filename = os.path.join(folder_path, f"{folder_name}.sumy")
    with open(summary_filename, "w", encoding="utf-8") as f:
        f.write(final_summary)
    
    print(f"Summary written for {folder_path} to {summary_filename}")

def read_any_root_sumy(repo_path):
    """
    Reads and returns the content of any .sumy file found in the root directory.
    """
    for entry in os.listdir(repo_path):
        if entry.endswith(".sumy"):
            with open(os.path.join(repo_path, entry), "r", encoding="utf-8") as f:
                return f.read()
    print("No .sumy file found in the root directory.")
    return None

if __name__ == '__main__':
    # Replace this with your actual repository path
    repo_path = "/Users/reva/Documents/geek_projects/dungbeetle"

    start_time = time.time()
    traverse_and_summarize(repo_path, repo_path)
    end_time = time.time()

    print(f"Time taken to execute the summary calculation in serial : {end_time - start_time:.6f} seconds")

    summary_output = read_any_root_sumy(repo_path)
    if summary_output:
        print("\n--- Root Project Summary ---\n")
        print(summary_output)

Summary written for /Users/reva/Documents/geek_projects/dungbeetle/cmd to /Users/reva/Documents/geek_projects/dungbeetle/cmd/cmd.sumy
Summary written for /Users/reva/Documents/geek_projects/dungbeetle/internal/dbpool to /Users/reva/Documents/geek_projects/dungbeetle/internal/dbpool/dbpool.sumy
Summary written for /Users/reva/Documents/geek_projects/dungbeetle/internal/core to /Users/reva/Documents/geek_projects/dungbeetle/internal/core/core.sumy
Summary written for /Users/reva/Documents/geek_projects/dungbeetle/internal/resultbackends/sqldb to /Users/reva/Documents/geek_projects/dungbeetle/internal/resultbackends/sqldb/sqldb.sumy
Summary written for /Users/reva/Documents/geek_projects/dungbeetle/internal/resultbackends to /Users/reva/Documents/geek_projects/dungbeetle/internal/resultbackends/resultbackends.sumy
Summary written for /Users/reva/Documents/geek_projects/dungbeetle/internal to /Users/reva/Documents/geek_projects/dungbeetle/internal/internal.sumy
Summary written for /Users/r